In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler


# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")

data = data.drop("date", axis=1)

data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()

data = data.astype('float32')


# Keep data until 31.08.2023
data = data.iloc[:10731]

#print(data['open'].dtype)
#print(data.shape)

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Now x_data and y_data are pandas DataFrames/Series, respectively

x_data.tail(1)




# size of the window for data preparation
split_window_size = 20

# Initialize lists to store training and temporary sets
x_train_list, y_train_list, x_temp_list, y_temp_list = [], [], [], []

# Iterate through the data with the specified window size
for i in range(0, len(x_data) - split_window_size, split_window_size + 1):
    x_train_temp = x_data.iloc[i:i+split_window_size+1]
    y_train_temp = y_data.iloc[i:i+split_window_size+1]

    # Separate the last row for the temporary set
    x_train = x_train_temp.iloc[:-1]
    y_train = y_train_temp.iloc[:-1]

    x_temp = x_train_temp.iloc[-1:]
    y_temp = y_train_temp.iloc[-1:]

    x_train_list.append(x_train)
    y_train_list.append(y_train)
    x_temp_list.append(x_temp)
    y_temp_list.append(y_temp)

# Concatenate the lists into pandas DataFrames
x_train = pd.concat(x_train_list)
y_train = pd.concat(y_train_list)
x_temp = pd.concat(x_temp_list)
y_temp = pd.concat(y_temp_list)

# print(y_train.head(50))
x_temp_train, x_temp_val, y_temp_train, y_temp_val = train_test_split(x_temp, y_temp, test_size=0.2, random_state=42)


# Split x_temp and y_temp into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42)


"""
# Print the last 5 rows of x_data
print("Last 5 rows of x_data:")
print(x_data.tail(5))

# Print the last 5 rows of x_train
print("\nLast 25 rows of x_train:")
print(x_train.tail(25))

print("\nLast 3 rows of y_train:")
print(y_temp.tail(3))
"""




scaler = MinMaxScaler()

x_train_normalized = scaler.fit_transform(x_train)
x_val_normalized = scaler.transform(x_val)
x_test_normalized = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(x_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(x_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)




"""
# x_train_tensor inverse

x_test_original = scaler.inverse_transform(x_train_tensor.numpy())
print("\nFirst row of x_test_original:")
print(x_test_original[0])

print("\nFirst row of x_train:")
print(x_train.head(1))



print("\nLast row of x_test_original:")
print(x_test_original[-1])

print("\nLast row of x_train:")
print(x_train.tail(1))
"""



'\n# x_train_tensor inverse\n\nx_test_original = scaler.inverse_transform(x_train_tensor.numpy())\nprint("\nFirst row of x_test_original:")\nprint(x_test_original[0])\n\nprint("\nFirst row of x_train:")\nprint(x_train.head(1))\n\n\n\nprint("\nLast row of x_test_original:")\nprint(x_test_original[-1])\n\nprint("\nLast row of x_train:")\nprint(x_train.tail(1))\n'

In [2]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, learning_rate, window_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

In [3]:
import itertools

# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [1, 2, 4, 8, 16, 32]
num_layers_list = [1, 2, 3, 4, 5, 6 ,7, 8]
learning_rates = [0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001 ]
window_sizes = [1, 2, 5, 10, 15, 20, 25, 30]
num_epochs = 250

# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

best_validation_loss = float('inf')
best_test_loss = float('inf')
best_hyperparameters = None

# Iterate over each hyperparameter combination
for hyperparameters in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size = hyperparameters

    print()
    print("Hyperparameters:", hyperparameters)

    # Initialize the model with current hyperparameters
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validate the model
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)

            avg_train_loss = running_loss / len(x_train_tensor)
            avg_val_loss = val_loss / len(x_val_tensor)

            print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}')

    # Evaluate the model on test data
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    avg_test_loss = test_loss / len(x_test_tensor)
    print(f'Test Loss: {avg_test_loss}')

    # Update best hyperparameters if test loss is lower
    if avg_test_loss < best_test_loss:
        best_test_loss = avg_test_loss
        best_hyperparameters = hyperparameters

print()
print("Best Hyperparameters:", best_hyperparameters)
print("Best Test Loss:", best_test_loss)



Hyperparameters: (7, 1, 1, 0.05, 1)


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/250], Training Loss: 5783.797695039127, Validation Loss: 24890.080078125
Epoch [2/250], Training Loss: 6629.436997168329, Validation Loss: 21181.5859375
Epoch [3/250], Training Loss: 3588.818219767616, Validation Loss: 13155.716796875
Epoch [4/250], Training Loss: 2080.035836915165, Validation Loss: 10396.0849609375
Epoch [5/250], Training Loss: 1595.4826714875335, Validation Loss: 10820.595703125
Epoch [6/250], Training Loss: 1581.5505834016008, Validation Loss: 13306.0634765625
Epoch [7/250], Training Loss: 1561.5162595959625, Validation Loss: 11396.19140625
Epoch [8/250], Training Loss: 1526.6856800041246, Validation Loss: 13680.48828125
Epoch [9/250], Training Loss: 2037.7854366671545, Validation Loss: 17058.76171875
Epoch [10/250], Training Loss: 2274.553359637241, Validation Loss: 11469.751953125
Epoch [11/250], Training Loss: 3374.038573829839, Validation Loss: 11277.0029296875
Epoch [12/250], Training Loss: 2196.8512393495726, Validation Loss: 11775.0859375
Epoch [13/2

KeyboardInterrupt: 